In [1]:
ls

1                              LIG-Doctor/                    strataboy.ipynb
applicationcluster/            LoadAndTrain.ipynb             test.ipynb
batchFiles/                    majup*                         train_eval.ipynb
bin/                           mimic-iv-2.2/                  Training.ipynb
Clinical-GAN/                  OutFiles/                      Training.py
CSS/                           PatientTrajectoryForecasting/  Untitled.ipynb
data-gen-mimic-iv_works.ipynb  PrepareData.ipynb              wandb/
DataImportants/                RisingThunder.ipynb            zifSnXUB
doctorai/                      seeker.ipynb
final/                         slurm-5653.out


In [2]:
cd PatientTrajectoryForecasting

/home/sifal.klioui/PatientTrajectoryForecasting


In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from dataclasses import dataclass, asdict
import yaml
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from utils.train import train_epoch, evaluate, get_data_loaders
import wandb
import argparse
import os
from model import Seq2SeqTransformer
from utils.eval import get_sequences, mapk
import warnings
# currently getting warnings because of mask datatypes, you might wanna change this not installing from environment.yml
warnings.filterwarnings("ignore") 

In [8]:
@dataclass
class DataConfig:
    strategy : str = 'SDP'
    seed : int = 89957
    test_size : float = 0.05
    valid_size : float = 0.10
    predict_procedure : bool = None
    predict_drugs : bool = None
    input_max_length :int = 448
    target_max_length :int = 64
    source_vocab_size : int = None
    target_vocab_size : int = None
    target_pad_id : int = 0
    source_pad_id : int = 0

In [31]:
@dataclass
class Config:
    positional_encoding : bool = True
    num_encoder_layers: int = 6
    num_decoder_layers: int = 6
    nhead: int = 8
    emb_size: int = 1024
    ffn_hid_dim: int = 1024
    dropout: float = 0.1
    train_batch_size: int = 64
    eval_batch_size: int = 128
    learning_rate: float = 0.0001
    warmup_start: float = 5
    num_train_epochs: int = 45
    warmup_epochs: int = None
    label_smoothing : float = 0.05
    scheduler : str = 'StepLR'
    factor : float = 0.1
    patience : int = 5
    T_0 : int = 10
    T_mult : int = 2
    step_size : int = 10
    gamma : float = 0.1

In [18]:
cd ..

/home/sifal.klioui


In [19]:
config = Config()
data_config = DataConfig()
train_dataloader, val_dataloader, test_dataloader, src_tokens_to_ids, tgt_tokens_to_ids, _, data_and_properties  = get_data_loaders(**asdict(data_config))

data_config.source_vocab_size = data_and_properties['embedding_size_source']
data_config.target_vocab_size = data_and_properties['embedding_size_target']
data_config.target_pad_id = tgt_tokens_to_ids['PAD']
data_config.source_pad_id = src_tokens_to_ids['PAD']

new_to_old_ids_source file not availble, mapping is the same as the old on


In [20]:
cd PatientTrajectoryForecasting/

/home/sifal.klioui/PatientTrajectoryForecasting


In [83]:
wandb.init(
      # Set the project where this run will be logged
      project="PTF_SDP_D", 
      # Track hyperparameters and run metadata
      config=asdict(config))

In [81]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transformer = Seq2SeqTransformer(config.num_encoder_layers, config.num_decoder_layers, config.emb_size,
                             config.nhead, data_config.source_vocab_size,
                             data_config.target_vocab_size, config.ffn_hid_dim,
                            config.dropout,
                            config.positional_encoding)

print(f'number of params: {sum(p.numel() for p in transformer.parameters()) / 1e6 : 0.2f}m')

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    transformer = nn.DataParallel(transformer)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index= data_config.target_pad_id, label_smoothing = config.label_smoothing)

optimizer = torch.optim.AdamW(transformer.parameters(), lr=config.learning_rate)

# Select the scheduler based on configuration
if config.scheduler == 'StepLR':
    scheduler = lr_scheduler.StepLR(optimizer, step_size=config.step_size, gamma=config.gamma)
elif config.scheduler == 'ReduceLROnPlateau':
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=config.factor, patience=config.patience)
elif config.scheduler == 'CosineAnnealingWarmRestarts':
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=config.T_0, T_mult=config.T_mult)

number of params:  106.92m


In [84]:
ks = [20,40,72]

In [ ]:
try:    
    for epoch in range(1,config.num_train_epochs):
        val_mapk = {}
        train_loss = train_epoch(transformer,  optimizer, train_dataloader, loss_fn, data_config.source_pad_id, data_config.target_pad_id, DEVICE)
        val_loss =  evaluate(transformer, val_dataloader, loss_fn, data_config.source_pad_id, data_config.target_pad_id, DEVICE)
        pred_trgs, targets =  get_sequences(transformer, val_dataloader, data_config.source_pad_id, tgt_tokens_to_ids, max_len = 72, DEVICE = DEVICE)
        if pred_trgs:
            val_mapk = {f"val_map@{k}": mapk(targets, pred_trgs, k) for k in ks}
            wandb.log({"Epoch": epoch, "train_loss": train_loss,"val_loss":val_loss, "lr" : optimizer.param_groups[0]['lr'], **val_mapk})
        if config.scheduler :
        # Step the scheduler based on its type
            if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
except Exception as e:
        wandb.log({"error": str(e)})
        raise e

train:  71%|███████   | 156/220 [04:30<01:50,  1.73s/it]

In [ ]:
finally:
    wandb.finish()

/home/sifal.klioui


In [15]:
len(pred_trgs)

3303

In [17]:
val_mapk

{'val_map@20': 0.48360119523507766,
 'val_map@40': 0.48428558322105636,
 'val_map@60': 0.48382014672449825}

In [16]:
val_mapk = {f"val_map@{k}": mapk(targets, pred_trgs, k) for k in ks}

In [22]:
if config.scheduler :
# Step the scheduler based on its type
    if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
        scheduler.step(val_loss)
    else:
        scheduler.step()

In [ ]:
mapk

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transformer = Seq2SeqTransformer(config.num_encoder_layers, config.num_decoder_layers, config.emb_size,
                             config.nhead, data_config.source_vocab_size,
                             data_config.target_vocab_size, config.ffn_hid_dim)

print(f'number of params: {sum(p.numel() for p in transformer.parameters())}')

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    transformer = nn.DataParallel(transformer)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index= data_config.target_pad_id, label_smoothing = config.label_smoothing)

optimizer = torch.optim.AdamW(transformer.parameters(), lr=config.learning_rate)

# Select the scheduler based on configuration
if config.scheduler == 'StepLR':
    scheduler = lr_scheduler.StepLR(optimizer, step_size=config.step_size, gamma=config.gamma)
elif config.scheduler == 'ReduceLROnPlateau':
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=config.factor, patience=config.patience)
elif config.scheduler == 'CosineAnnealingWarmRestarts':
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=config.T_0, T_mult=config.T_mult)

# add wandb loss logging
for epoch in range(config.num_train_epochs):
    val_mapk = {}
    train_loss = train_epoch(transformer,  optimizer, train_dataloader, loss_fn, data_config.source_pad_id, data_config.target_pad_id, DEVICE)
    val_loss =  evaluate(transformer, val_dataloader, loss_fn, data_config.source_pad_id, data_config.target_pad_id, DEVICE)
    pred_trgs, targets =  get_sequences(transformer, val_dataloader, data_config.source_pad_id, tgt_tokens_to_ids, max_len = 72, DEVICE = DEVICE)
    if pred_trgs:
        val_mapk = {f"val_map@{k}": mapk(targets, pred_trgs, k) for k in ks}
        print('aa') 
        break
        wandb.log({"Epoch": epoch, "train_loss": train_loss,"val_loss":val_loss, "lr" : optimizer.param_groups[0]['lr'], **val_mapk})
    if config.scheduler :
    # Step the scheduler based on its type
        if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
            scheduler.step(val_loss)
        else:
            scheduler.step()


In [31]:
cd sifal.klioui

/home/sifal.klioui


In [32]:

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transformer = Seq2SeqTransformer(config.num_encoder_layers, config.num_decoder_layers, config.emb_size,
                             config.nhead, data_config.source_vocab_size,
                             data_config.target_vocab_size, config.ffn_hid_dim)

print(f'number of params: {sum(p.numel() for p in transformer.parameters())}')

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    transformer = nn.DataParallel(transformer)

transformer = transformer.to(DEVICE)

new_to_old_ids_source file not availble, mapping is the same as the old on
number of params: 106922688


In [15]:
if __name__ == '__main__':

    config = Config()
    data_config = DataConfig()
    train_dataloader, val_dataloader, test_dataloader, src_tokens_to_ids, tgt_tokens_to_ids, _, data_and_properties  = get_data_loaders(**asdict(data_config))
    
    data_config.source_vocab_size = data_and_properties['embedding_size_source']
    data_config.target_vocab_size = data_and_properties['embedding_size_target']
    data_config.target_pad_id = tgt_tokens_to_ids['PAD']
    data_config.source_pad_id = src_tokens_to_ids['PAD']
    exit()
    wandb.init(
      # Set the project where this run will be logged
      project="PTF_SDP_D", 
      # Track hyperparameters and run metadata
      config=asdict(config))
    try:
        train_transformer(config, data_config, train_dataloader, val_dataloader, tgt_tokens_to_ids)
    except Exception as e:
        wandb.log({"error": str(e)})
        raise e
    finally:
        wandb.finish()

new_to_old_ids_source file not availble, mapping is the same as the old on


wandb: Currently logged in as: sifalklioui. Use `wandb login --relogin` to force relogin


/public/conda/Miniconda/envs/pytorch-2.2/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


number of params: 106922688


train:   0%|          | 0/208 [00:00<?, ?it/s]/public/conda/Miniconda/envs/pytorch-2.2/lib/python3.11/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched src_key_padding_mask and mask is deprecated. Use same type for both instead.
  warnings.warn(
/public/conda/Miniconda/envs/pytorch-2.2/lib/python3.11/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
train:   4%|▍         | 9/208 [00:22<08:26,  2.55s/it]


KeyboardInterrupt: 